In [1]:
import os
import sys

# Add project root to path
PROJECT_ROOT = os.path.abspath(os.path.join(".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)

PLOTS_DIR = "../plots"
RESULTS_DIR = "../results"

os.makedirs(PLOTS_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

In [2]:
from src.data_io import load_csv

In [3]:
from src.strategy import generate_signals, calculate_metrics

In [4]:
import joblib

In [5]:
import pandas as pd

In [6]:
# 06. Strategy Backtest

In [7]:
df = load_csv('data/processed/nifty_features_5min.csv')

Loaded 126 rows from F:\antigravity\klypto assignment\data/processed/nifty_features_5min.csv


In [8]:
# Predict Regime

In [9]:
df = df.dropna()

In [10]:
feats = ['avg_iv', 'iv_spread', 'ema_5']

In [11]:
valid_feats = [f for f in feats if f in df.columns]

In [12]:
if not valid_feats: valid_feats = ['returns']

In [13]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from hmmlearn.hmm import GaussianHMM

# 1. Prepare feature matrix
X = df[valid_feats].copy()
X = X.replace([np.inf, -np.inf], np.nan)
X = X.dropna()

# 2. Minimal data check
if X.shape[0] < 20:
    raise RuntimeError("Too few samples for regime detection")

# 3. Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 4. Create and FIT HMM on the SAME data
model = GaussianHMM(
    n_components=3,
    covariance_type="diag",   # critical for stability
    n_iter=200,
    random_state=42
)

model.fit(X_scaled)

# 5. Predict regimes
regimes = model.predict(X_scaled)

# 6. Align back to dataframe
df_regime = df.loc[X.index].copy()
df_regime["regime"] = regimes

# 7. Validation output
print("Regime counts:")
print(df_regime["regime"].value_counts())


Regime counts:
regime
2    13
0     9
1     9
Name: count, dtype: int64


In [14]:
df['regime'] = regimes

In [15]:
# Map Regime

In [16]:
mapping = {0:0, 1:1, 2:-1} # Simplified default

In [17]:
df['regime'] = df['regime'].map(mapping)

In [18]:
# Signals

In [19]:
res = generate_signals(df)

In [20]:
# Metrics

In [21]:
metrics = calculate_metrics(res)
pd.DataFrame([metrics]).to_csv(f"{RESULTS_DIR}/strategy_metrics.csv", index=False)

In [22]:
print(metrics)

{'Sharpe': np.float64(-32.16), 'Win Rate': 0.42, 'Max Drawdown': np.float64(-0.03), 'Calmar': np.float64(-0.79), 'Total Return': np.float64(-0.02)}


In [23]:
# Plot Equity Curve
import matplotlib.pyplot as plt

# Check if 'cumulative_returns' is directly in res, or if we need to calculate it
# generate_signals returns dataframe with 'returns' and 'strategy_returns' usually.
# Let's assume we need to calculate cumulative returns if not present.
if 'strategy_returns' in res.columns:
    res['cumulative_returns'] = (1 + res['strategy_returns']).cumprod()
elif 'returns' in res.columns: # Fallback
    res['cumulative_returns'] = (1 + res['returns']).cumprod()

plt.figure(figsize=(10, 6))
if 'cumulative_returns' in res.columns:
    plt.plot(res['cumulative_returns'], label='Equity Curve')
    plt.title('Strategy Equity Curve')
    plt.xlabel('Time')
    plt.ylabel('Cumulative Returns')
    plt.legend()
    plt.grid(True)
    plt.savefig(f"{PLOTS_DIR}/equity_curve.png")
    plt.close()